In [0]:
catalog_name = 'jon_cheung'
schema_name = 'vizio_poc'
vectors_per_shard= 100_000_000

base_num = vectors_per_shard // 1_000_000
base_string = str(base_num) + 'M'

In [0]:
# Create chunked data
sdf = spark.read.table('jonathan_mcfadden.vizio_poc.audio_test').select("id", "list_col")
num_chunks = (sdf.count() + vectors_per_shard - 1) // vectors_per_shard
chunks = [sdf.filter((sdf.id >= i * vectors_per_shard) & (sdf.id < (i + 1) * vectors_per_shard)) for i in range(num_chunks)]

In [0]:
import os
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import catalog

w = WorkspaceClient()

for shard_number, shard_name in enumerate([f'audio_{base_string}_chunk_shard_{value}' for value in range(1, int(num_chunks)+1)]):
  
  parquet_write_path = f'/Volumes/{catalog_name}/{schema_name}/{shard_name}'
  # Create a volume for the shard if that doesn't exist
  if not os.path.exists(parquet_write_path):
    created_volume = w.volumes.create(catalog_name=catalog_name,
                                      schema_name=schema_name,
                                      name=shard_name,
                                      volume_type=catalog.VolumeType.MANAGED
                                      )
  else:
    print(f"Volume {shard_name} already exists")
  
  # Populate the volume with parquet if it doesn't exist. We write to parquet because LanceDB doesn't support Spark DataFrames yet. 
  if not os.listdir(parquet_write_path):
    chunks[shard_number].write.mode('overwrite').parquet(parquet_write_path)
  else:
    print(f"Parquet directory {parquet_write_path} already has data in it.")
